## PvP game report exploration

In this note we download some pvp game reports and try to analyze them.

### Parametes

In [132]:
import requests as rq
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


In [7]:
apiKey = "cd2254b7d1d24127abb7a98a3b0ea10e"
HEADERS = {"X-API-Key": apiKey}
BUNGIE = "https://www.bungie.net/platform/Destiny/"
AccountType = "2"

In [8]:
mid = '4611686018428698313'

In [9]:
cid = '2305843009250714821'

### Getting game instance id

In [10]:
url = BUNGIE + "Stats/ActivityHistory/" + AccountType + '/' + mid + '/' + cid + '/?mode=AllPvP&count=10'

In [11]:
r = rq.get(url, headers=HEADERS)

In [12]:
res = r.json()['Response']

In [13]:
acts = res['data']['activities']

In [14]:
PvPinstanceId = [a['activityDetails']['instanceId'] for a in acts]

In [15]:
PvPinstanceId

[u'6384227424',
 u'6384205568',
 u'6384180296',
 u'6384156839',
 u'6384137947',
 u'6384113384',
 u'6384093482',
 u'6384061436',
 u'5672390977',
 u'5672314277']

### Getting game reports

In [16]:
def getReport(instanceId):
    url = BUNGIE + '/Stats/PostGameCarnageReport/' + instanceId + '/'
    r = rq.get(url, headers=HEADERS)
    return r.json()['Response']['data']

In [17]:
reports = [getReport(iid) for iid in PvPinstanceId]

In [26]:
reports[0]['entries'][0]['extended']['values']['weaponKillsSuper']

{u'basic': {u'displayValue': u'6', u'value': 6.0}}

In [27]:
reports[0]['entries'][0]['extended']['values']['weaponKillsRelic']

KeyError: 'weaponKillsRelic'

In [39]:
for r in reports:
    for p in r['entries']:
        print "player Class", p['player']['characterClass']
        # if 'weaponKillsSuper' in p['extended']['values']:
        #    print 'SuperKill', p['extended']['values']['weaponKillsSuper']['basic']['value']
        if 'weaponKillsRelic' in p['extended']['values']:
            print 'RelicKill', p['extended']['values']['weaponKillsRelic']['basic']['value']

player Class Titan
player Class Warlock
player Class Hunter
player Class Titan
player Class Hunter
player Class Warlock
player Class Hunter
player Class Titan
player Class Hunter
player Class Warlock
player Class Hunter
player Class Warlock
player Class Hunter
player Class Warlock
player Class Warlock
player Class Titan
player Class Warlock
player Class Titan
player Class Hunter
player Class Hunter
player Class Hunter
player Class Titan
player Class Titan
player Class Warlock
player Class Warlock
player Class Titan
player Class Hunter
player Class Hunter
player Class Warlock
player Class Hunter
player Class Hunter
player Class Warlock
player Class Hunter
player Class Warlock
player Class Warlock
player Class Hunter
player Class Warlock
player Class Hunter
player Class Hunter
player Class Hunter
player Class Hunter
player Class Warlock
player Class Hunter
player Class Hunter
player Class Hunter
player Class Warlock
player Class Warlock
player Class Hunter
player Class Hunter
player Clas

In [37]:
reports[0]['entries'][0]['player']['characterClass']

u'Titan'

### No relic kills identified yet. Good news. Need more data to verify.

In [40]:
url = BUNGIE + "Stats/ActivityHistory/" + AccountType + '/' + mid + '/' + cid + '/?mode=AllPvP&count=100'

In [41]:
r = requests.get(url, headers=HEADERS)

In [42]:
PvPinstanceId = [a['activityDetails']['instanceId'] for a in r.json()['Response']['data']['activities']]

In [44]:
reports = [getReport(iid) for iid in PvPinstanceId]

In [53]:
from pandas.io.json import json_normalize

In [102]:
def convertToDF(report):
    df = json_normalize(report['entries'])
    df = df.assign(instanceId = pd.Series([report['activityDetails']['instanceId']] * len(df)))
    return df

In [105]:
df = pd.concat([convertToDF(r) for r in reports])

In [109]:
list(df)

[u'characterId',
 u'extended.values.allMedalsEarned.basic.displayValue',
 u'extended.values.allMedalsEarned.basic.value',
 u'extended.values.allMedalsScore.basic.displayValue',
 u'extended.values.allMedalsScore.basic.value',
 u'extended.values.allParticipantsCount.basic.displayValue',
 u'extended.values.allParticipantsCount.basic.value',
 u'extended.values.allParticipantsScore.basic.displayValue',
 u'extended.values.allParticipantsScore.basic.value',
 u'extended.values.allParticipantsTimePlayed.basic.displayValue',
 u'extended.values.allParticipantsTimePlayed.basic.value',
 u'extended.values.assistsAgainstPlayerHunter.basic.displayValue',
 u'extended.values.assistsAgainstPlayerHunter.basic.value',
 u'extended.values.assistsAgainstPlayerTitan.basic.displayValue',
 u'extended.values.assistsAgainstPlayerTitan.basic.value',
 u'extended.values.assistsAgainstPlayerWarlock.basic.displayValue',
 u'extended.values.assistsAgainstPlayerWarlock.basic.value',
 u'extended.values.averageKillDistance.

In [128]:
weaponKillsColNames = [s for s in list(df) if s.startswith('extended.values.weapon') and s.endswith('.value') and s!= u'extended.values.weaponBestType.basic.value' ]

In [159]:
weaponStats = pd.DataFrame(df[weaponKillsColNames].sum())

In [160]:
weaponStats = weaponStats.reset_index()

In [164]:
weaponStats = weaponStats.assign(weaponType = pd.Series([s.split('.')[2] for s in weaponStats['index']]))

In [165]:
del weaponStats['index']

In [167]:
weaponStats.columns = ['kills', 'weaponType']

In [176]:
weaponStats.sort_values(by = 'kills', ascending= False)

,kills,weaponType
21,1998.000000,weaponKillsSuper
17,1922.000000,weaponKillsScoutRifle
18,1416.000000,weaponKillsShotgun
5,1377.000000,weaponKillsMelee
0,1367.000000,weaponKillsAutoRifle
20,1276.000000,weaponKillsSniper
3,1116.000000,weaponKillsHandCannon
15,1032.000000,weaponKillsPulseRifle
2,928.000000,weaponKillsGrenade
28,894.000000,weaponPrecisionKillsScoutRifle


In [177]:
dfGames = json_normalize(reports)

In [179]:
del dfGames['entries']

In [180]:
dfGames

,activityDetails.activityTypeHashOverride,activityDetails.instanceId,activityDetails.isPrivate,activityDetails.mode,activityDetails.referenceId,period,teams
0,3846426416,6384227424,False,10,3292667877,2017-04-28T03:45:33Z,[{u'standing': {u'basic': {u'displayValue': u'...
1,3846426416,6384205568,False,10,4163254808,2017-04-28T03:31:22Z,[{u'standing': {u'basic': {u'displayValue': u'...
2,3846426416,6384180296,False,10,3848655103,2017-04-28T03:19:31Z,[{u'standing': {u'basic': {u'displayValue': u'...
3,3846426416,6384156839,False,10,3292667877,2017-04-28T03:06:39Z,[{u'standing': {u'basic': {u'displayValue': u'...
4,3846426416,6384137947,False,10,1719392441,2017-04-28T02:57:38Z,[{u'standing': {u'basic': {u'displayValue': u'...
5,3846426416,6384113384,False,10,2680821721,2017-04-28T02:46:22Z,[{u'standing': {u'basic': {u'displayValue': u'...
6,3846426416,6384093482,False,10,4105918304,2017-04-28T02:35:09Z,[{u'standing': {u'basic': {u'displayValue': u'...
7,3846426416,6384061436,False,10,1851417512,2017-04-28T02:22:45Z,[{u'standing': {u'basic': {u'displayValue': u'...
8,1860850614,5672390977,False,34,1709749894,2016-10-05T20:31:27Z,[{u'standing': {u'basic': {u'displayValue': u'...
9,1860850614,5672314277,False,34,3292667877,2016-10-05T20:16:51Z,[{u'standing': {u'basic': {u'displayValue': u'...


In [183]:
dfGames.groupby('activityDetails.mode').count()

,activityDetails.activityTypeHashOverride,activityDetails.instanceId,activityDetails.isPrivate,activityDetails.referenceId,period,teams
activityDetails.mode,,,,,,
10,12,12,12,12,12,12
11,2,2,2,2,2,2
12,29,29,29,29,29,29
13,1,1,1,1,1,1
23,3,3,3,3,3,3
24,35,35,35,35,35,35
34,18,18,18,18,18,18
